<a href="https://colab.research.google.com/github/PritishRane/VScode_23-11/blob/main/dose_calculation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
from warnings import filterwarnings
filterwarnings('ignore')

In [29]:
import pandas as pd

In [30]:
df = pd.read_csv('Crcldose.csv', encoding='ISO-8859-1')

In [31]:
df

,Drug,Range,Recommended Dose
0,Levofloxacin,> 50,500 mg every 24h
1,Levofloxacin,3050,250 mg every 24h
2,Levofloxacin,< 30,250 mg every 48h
3,Vancomycin,> 60,15 mg/kg every 12h
4,Vancomycin,3060,15 mg/kg every 24h
...,...,...,...
92,Delafloxacin,? 30,450 mg PO every 12h
93,Delafloxacin,< 30,200 mg IV every 12h (PO use not recommended)
94,Oritavancin,Any,No adjustment needed
95,Dalbavancin,Any,No adjustment needed


In [32]:
replacements = {
    '–': '-',    # en dash to hyphen
    '—': '-',    # em dash to hyphen
    '−': '-',    # minus to hyphen
    '': '-',    # corrupted dash (cp1252 issue)
    '≥': '>=',
    '≤': '<=',
    '?': '>=',   # assuming ? is used for ≥ in your file
    '�': '',     # unknown character placeholder
}

In [33]:
for col in df.columns:
    if df[col].dtype == 'object':
        for bad, good in replacements.items():
            df[col] = df[col].str.replace(bad, good, regex=False)


In [34]:
df

,Drug,Range,Recommended Dose
0,Levofloxacin,> 50,500 mg every 24h
1,Levofloxacin,30-50,250 mg every 24h
2,Levofloxacin,< 30,250 mg every 48h
3,Vancomycin,> 60,15 mg/kg every 12h
4,Vancomycin,30-60,15 mg/kg every 24h
...,...,...,...
92,Delafloxacin,>= 30,450 mg PO every 12h
93,Delafloxacin,< 30,200 mg IV every 12h (PO use not recommended)
94,Oritavancin,Any,No adjustment needed
95,Dalbavancin,Any,No adjustment needed


In [35]:
# Step 1: CrCl Calculation
def calculate_crcl(age, weight, sex, serum_creatinine):
    factor = 0.85 if sex.upper() == 'F' else 1.0
    crcl = ((140 - age) * weight * factor) / (72 * serum_creatinine)
    return round(crcl, 2)

In [36]:
# Step 2: Clean range and check if CrCl fits
def crcl_in_range(crcl, range_str):
    range_str = range_str.strip()
    if range_str.lower() == "any":
        return True
    elif '>' in range_str:
        value = float(range_str.replace('>', '').strip())
        return crcl > value
    elif '<' in range_str:
        value = float(range_str.replace('<', '').strip())
        return crcl < value
    elif '-' in range_str:
        parts = range_str.split('-')
        low, high = float(parts[0].strip()), float(parts[1].strip())
        return low <= crcl <= high
    elif '>=' in range_str:
        value = float(range_str.replace('>=', '').strip())
        return crcl >= value
    elif '<=' in range_str:
        value = float(range_str.replace('<=', '').strip())
        return crcl <= value
    return False

In [37]:
# Step 3: Load CSV and find correct dose
def convert_mg_per_kg_to_mg(dose_str, weight):
    import re
    match = re.search(r'(\d+)\s*mg/kg', dose_str)
    if match:
        mg_per_kg = int(match.group(1))
        total_dose = mg_per_kg * weight
        # Replace "mg/kg" with the calculated mg value
        new_dose_str = re.sub(r'\d+\s*mg/kg', f'{total_dose} mg', dose_str)
        return new_dose_str
    return dose_str

In [38]:
def get_recommended_dose(drug, crcl, weight, df):
    drug = drug.strip().title()
    matching_rows = df[df['Drug'].str.title() == drug]

    if matching_rows.empty:
        return f"⚠️ Drug '{drug}' not found in dose table."

    for _, row in matching_rows.iterrows():
        if crcl_in_range(crcl, row['Range']):
            dose = convert_mg_per_kg_to_mg(row['Recommended Dose'], weight)
            return f"✅ {drug} dose for CrCl {crcl} mL/min: {dose}"

    return f"⚠️ No matching dose range found for CrCl {crcl} mL/min for drug {drug}."

In [39]:
age = 65
weight = 70  # kg
sex = 'F'
serum_creatinine = 1.2
drug = 'Vancomycin'

In [40]:
crcl = calculate_crcl(age, weight, sex, serum_creatinine)

In [41]:
crcl

51.65

In [43]:
result = get_recommended_dose(drug, crcl, weight, df)

In [44]:
result

'✅ Vancomycin dose for CrCl 51.65 mL/min: 1050 mg every 24h'